In [3]:
import os
import pathlib
import tifffile

# Convert 3D TIFF file (x,y,time) to single files for each frame and save them to a folder for each movie.

input_path = pathlib.Path('/Users/domchom/Desktop/Cell_profiler_test/Bement-PatternAnalysis/dom_testing/images')
output_path = pathlib.Path('/Users/domchom/Desktop/Cell_profiler_test/Bement-PatternAnalysis/dom_testing/images_new')

for file_path in input_path.glob('*.tif'):
    try:
        # Load TIFF file and extract metadata
        image = tifffile.imread(file_path)
        with tifffile.TiffFile(file_path) as tif_file:
            metadata = tif_file.imagej_metadata
            num_channels = metadata.get('channels', 1)
            num_slices = metadata.get('slices', 1)
            num_frames = metadata.get('frames', 1)
            image = image.reshape(num_frames, num_slices, num_channels, image.shape[-1], image.shape[-2])
            
        # Save single-frame TIFF files to output path
        filename = os.path.splitext(os.path.basename(file_path))[0]
        subfolder_path = os.path.join(output_path, filename)
        os.makedirs(subfolder_path, exist_ok=True)
        
        for frame_idx, single_frame_array in enumerate(image):
            single_frame_path = os.path.join(subfolder_path, f'{filename}_frame_{frame_idx:04d}.tif')
            with tifffile.TiffWriter(single_frame_path) as tif:
                tif.save(single_frame_array[0, 0, :, :])
                    
    except Exception as e:
        print(f'Error processing {file_path}: {e}')
